In [ ]:
%cd "C:\\Users\\Administrator\\Desktop\\强化学习\\中证50指数增强\\"
path = "C:\\Users\\Administrator\\Desktop\\强化学习\\中证50指数增强\\"

import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import numpy as np
import random
import pandas as pd
#显示Dateframe所有列(参数设置为None代表显示所有行，也可以自行设置数字)
pd.set_option('display.max_columns',None)
#显示Dateframe所有行
pd.set_option('display.max_rows',None)
#设置Dataframe数据的显示长度，默认为50
pd.set_option('max_colwidth',200)
#不显示科学计数法
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from stock_portfoilo_env import *
from stable_baselines3 import *
from stable_baselines3.common.callbacks import *
from stable_baselines3.common.vec_env import *
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
import plotly.graph_objects as go

import talib
from tqdm import tqdm
import torch
import os
import time
import gym

# Batch backtesting of all the model files

In [ ]:
def read_model_parameters(folder, tmp_model):
    serial_num = []
    return_rate = []
    counter = 0
    for a in range(len(tmp_model)):
        if tmp_model[a] == '_':
            counter += 1
        else:
            pass
        
        if counter <= 0 and counter < 1 and tmp_model[a] != '_':
            serial_num.append(tmp_model[a])
        else:
            pass
        
        if counter >=1 and counter < 2 and tmp_model[a] != '_':
            return_rate.append(tmp_model[a])
        else:
            pass
        
    serial_num =  str(serial_num).replace(",","").replace("'","").replace("[","").replace("]","").replace(" ","")
    return_rate = str(return_rate).replace(",","").replace("'","").replace("[","").replace("]","").replace(" ","")
    tmp_params = np.load(folder+serial_num+'_'+return_rate+'.npy', allow_pickle=True)
    window_size = tmp_params[0]
    factors = tmp_params[1]
    std_rolling = tmp_params[2]
    return window_size, factors, std_rolling

In [ ]:
log_dir = path + 'best_models\\'
if os.path.exists(log_dir) == True:
    pass
else:
    os.makedirs(log_dir) 

files = os.listdir(log_dir) #得到文件夹下的所有文件名称
model_files = []
for i in range(len(files)):
    tmp_file = files[i]
    if '2023-' in tmp_file and 'RewardMethod1' in tmp_file:
        model_files.append(tmp_file)
    else:
        pass

len(model_files), model_files

In [ ]:
tmp = []
try:
    prev_bulk_backtest = pd.read_csv('bulk_backtest.csv').drop(columns='Unnamed: 0')
    tested_files = prev_bulk_backtest['model file'].to_list()
except:
    pass
tested_files = []
slippage = 2
commission_rate = 0.00015
tmp = []
for i in tqdm(range(len(model_files))):
    torch.cuda.empty_cache()
   #torch.cuda.set_device(0)
    tmp_model = model_files[i]
    if tmp_model in tested_files:
        print('Tested Model File, Skip')
        pass
    else:
        try:
            window_size, _, std_rolling = read_model_parameters(log_dir, tmp_model)
        except:
            continue
        tmp_test_obs = np.load(path+'处理后数据\\test_obs'+str(std_rolling)+'.npy', allow_pickle=True).astype('float32')
        tmp_test_price = np.load(path+'处理后数据\\test_price'+str(std_rolling)+'.npy', allow_pickle=True)
        tmp_test_symbols = np.load(path+'处理后数据\\test_symbols'+str(std_rolling)+'.npy', allow_pickle=True)
        stock_num = len(tmp_test_symbols[0])-1
        tmp_test_obs_1 = tmp_test_obs[:,-window_size:,:]
        
        print('')
        print('##########################################################')
        #print('Now Using GPU No.', torch.cuda.current_device())
        print(tmp_test_obs_1.shape)
        print(tmp_model)
        #with np.errstate(divide='ignore'):
        try:
            ###########################################################################################################################
            env = TradingEnv(obs=tmp_test_obs_1, price=tmp_test_price, symbols=tmp_test_symbols, both_pos=True, 
                            commission_rate=commission_rate, valid_folder=None, slippage=slippage, calculate_fund=True,
                            commission=None, test=False, valid=False, active_shares=True, cash_out_stop=False, 
                            returns=1, action_type=2, num_stock=stock_num)
            model = A2C.load(log_dir+tmp_model, device='cuda')
            obs, _ = env.reset()
            tmp_info_1 = None
            print('#################################### Out-of-sample Results ####################################')
            for i in range(tmp_test_obs_1.shape[0]-2):
                action, _states = model.predict(obs, deterministic=True)
                obs, rewards, dones, _, info = env.step(action)
                if dones == True:
                    tmp_info_1 = info
                else:
                    pass
            tmp.append([tmp_model, 'Out-Of-Sample', tmp_info_1['return rate'], tmp_info_1['maxdrawndown'], tmp_info_1['sharpe']])
        except:
            print('Error: ', tmp_model)
            pass
bulk_backtest = pd.DataFrame(tmp)
bulk_backtest.columns = ['model file', 'type', 'Return Rate', 'Maxdrawndown', 'sharpe']
try:
    bulk_backtest = bulk_backtest.append(prev_bulk_backtest)
except:
    pass

bulk_backtest.shape, bulk_backtest.head(5)

In [ ]:
try:
    bulk_backtest.to_csv('bulk_backtest.csv')
except:
    pass
bulk_backtest = pd.read_csv('bulk_backtest.csv').drop(columns='Unnamed: 0').sort_values(by='Return Rate', ascending=False).reset_index(drop=True)

bulk_backtest.head(10)

In [ ]:
def plot_backtest(reward, benchmark, split):
    networth = reward
    acc_networth = []
    for i in range(len(networth)):
        acc_networth.append((networth[i]-networth[0])/networth[0])
    print('账户总收益: ', np.around(acc_networth[-1]*100, 2), '%')
    try:
        print('基准指数总收益: ', np.around(benchmark[-1]*100, 2), '%')
        print('超额收益: ',  np.around((acc_networth[-1]-benchmark[-1])*100, 2), '%')
    except:
        pass
    %matplotlib inline
    plt.figure(figsize=(16,8))
    plt.axvline(split, color='red', label="测试集|验证集")
    plt.plot(acc_networth, label="账户收益率")
    plt.plot(benchmark, label='基准收益率')
    plt.xlabel("数据索引")
    plt.ylabel("收益率")
    plt.title("回测资金曲线")
    plt.legend(loc='best',frameon=False)
    plt.show()

# Load data

In [ ]:
tmp_model = bulk_backtest.iloc[0]['model file']
window_size, num_features, std_rolling = read_model_parameters(log_dir, tmp_model)

window_size, std_rolling

In [ ]:
tmp_test_obs = np.load(path+'处理后数据\\test_obs'+str(std_rolling)+'.npy', allow_pickle=True).astype('float32')
tmp_test_price = np.load(path+'处理后数据\\test_price'+str(std_rolling)+'.npy', allow_pickle=True)
tmp_test_symbols = np.load(path+'处理后数据\\test_symbols'+str(std_rolling)+'.npy', allow_pickle=True)

tmp_test_obs_1 = tmp_test_obs[:,-window_size:,:]

test_benchmark_prices = tmp_test_price[:,:-1].astype('float')
first_avg = np.mean(test_benchmark_prices[0])
test_benchmark_avg_prices = [first_avg]
for i in range(1, len(test_benchmark_prices)):
    tmp_avg = np.mean(test_benchmark_prices[i])
    test_benchmark_avg_prices.append(tmp_avg)
test_benchmark_prices = pd.Series(test_benchmark_avg_prices)
test_benchmark_prices['networth'] = (test_benchmark_prices-test_benchmark_prices.iloc[0])/test_benchmark_prices.iloc[0]
test_benchmark_prices = test_benchmark_prices['networth'].values
test_benchmark_dates = []
for i in range(tmp_test_price.shape[0]):
    test_benchmark_dates.append(tmp_test_price[i,-1])
test_benchmark_dates = np.asarray(test_benchmark_dates)

tmp_test_obs_1.shape, test_benchmark_dates.shape

# Load model and backtest (test data)

In [ ]:
env = TradingEnv(obs=tmp_test_obs_1, price=tmp_test_price, symbols=tmp_test_symbols, both_pos=True, 
                commission_rate=commission_rate, valid_folder=None, slippage=slippage, calculate_fund=True,
                commission=None, test=False, valid=False, active_shares=True, cash_out_stop=False, 
                returns=1, action_type=2, num_stock=stock_num)
model = A2C.load(log_dir+tmp_model, device='cpu')

obs, _ = env.reset()
reward_test = None
for i in tqdm(range(tmp_test_obs_1.shape[0]-2)):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, _, tmp_info = env.step(action)
test_account_his = tmp_info['account_history']

In [ ]:
print('样本外数据开始日期: ', tmp_test_price[0,-1], ', 样本外数据结束日期：', tmp_test_price[-1,-1])
plot_backtest(test_account_his, test_benchmark_prices, 252)